## Adhoc Analysis of GMail Messages using AWS Athena

As Glue Catalog Table is ready, one can perform analysis of GMail Messages using Athena Queries.
* Athena is a serverless query engine which can be used for ad-hoc analysis, reporting as well as processing the data via catalog tables created on top of files in s3.
* We can install libraries such as **pyathena** and **sqlalchemy** to run SQL queries from clients such as Jupyter. We can also use Athena Query Editor directly to run queries for ad-hoc analysis.

```
pip install pyathena
pip install sqlalchemy
```

In [1]:
from pyathena import connect
import pandas as pd
conn = connect(
    s3_staging_dir='s3://itversitydata/gmail_messages/',
    region_name='us-east-1'
)

In [2]:
messages = pd.read_sql("SELECT * FROM gmail_db.messages LIMIT 10", conn)
messages.head(2)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,threadid,labelids,snippet,payload,sizeestimate,historyid,internaldate
0,1802a73fd9486675,1802a73fd9486675,"[CATEGORY_PROMOTIONS, UNREAD, INBOX]","Save on games, PCs, accessories, Surface, and ...","{partid=, mimetype=multipart/alternative, file...",99511,11858200,1649979684000
1,1802a72c89ae6e8a,1802a72c89ae6e8a,"[UNREAD, CATEGORY_SOCIAL, INBOX]",LinkedIn Durga Gadiraju Sachchidanand Singh CS...,"{partid=, mimetype=multipart/alternative, file...",26631,11858274,1649979606000


In [3]:
pd.read_sql("SELECT count(*) FROM gmail_db.messages", conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,_col0
0,1055


In [4]:
pd.read_sql("SELECT min(id), max(id) FROM gmail_db.messages", conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,_col0,_col1
0,17fe27ee39d579d1,1802a73fd9486675


In [6]:
pd.read_sql('''
    SELECT id, 
        threadid, 
        headers.name,
        headers.value
    FROM gmail_db.messages
    CROSS JOIN unnest(payload.headers) as t(headers)
    LIMIT 10
''', conn)

,id,threadid,name,value
0,180107bd44ce81ba,180107bd44ce81ba,Delivered-To,dgadiraju@itversity.com
1,180107bd44ce81ba,180107bd44ce81ba,Received,by 2002:a4a:40c2:0:0:0:0:0 with SMTP id n185cs...
2,180107bd44ce81ba,180107bd44ce81ba,X-Received,by 2002:a05:6e02:19c9:b0:2ca:57a:9f62 with SMT...
3,180107bd44ce81ba,180107bd44ce81ba,ARC-Seal,i=1; a=rsa-sha256; t=1649543992; cv=none; ...
4,180107bd44ce81ba,180107bd44ce81ba,ARC-Message-Signature,i=1; a=rsa-sha256; c=relaxed/relaxed; d=google...
5,180107bd44ce81ba,180107bd44ce81ba,ARC-Authentication-Results,i=1; mx.google.com; dkim=pass header.i=@...
6,180107bd44ce81ba,180107bd44ce81ba,Return-Path,<3NwtSYhgKAK4UccUZSamPigWbSgg-bcfSdZmUccUZS.Qc...
7,180107bd44ce81ba,180107bd44ce81ba,Received,from mail-sor-f69.google.com (mail-sor-f69.goo...
8,180107bd44ce81ba,180107bd44ce81ba,Received-SPF,pass (google.com: domain of 3nwtsyhgkak4uccuzs...
9,180107bd44ce81ba,180107bd44ce81ba,Authentication-Results,mx.google.com; dkim=pass header.i=@googl...


In [8]:
pd.read_sql('''
    WITH q AS (
        SELECT id, 
            threadid, 
            headers.name,
            headers.value
        FROM gmail_db.messages
        CROSS JOIN unnest(payload.headers) as t(headers)
    ) SELECT count(*) FROM q
''', conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,_col0
0,30046


In [10]:
pd.read_sql('''
    WITH q AS (
        SELECT id, 
            threadid, 
            headers.name,
            headers.value
        FROM gmail_db.messages
        CROSS JOIN unnest(payload.headers) AS t(headers)
    ) SELECT count(*) 
    FROM q
    WHERE name = 'From'
''', conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,_col0
0,1144


In [11]:
pd.read_sql('''
    WITH q AS (
        SELECT id, 
            threadid, 
            headers.name,
            headers.value
        FROM gmail_db.messages
        CROSS JOIN unnest(payload.headers) AS t(headers)
    ) SELECT * 
    FROM q
    WHERE name = 'From'
    LIMIT 10
''', conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,threadid,name,value
0,180107bd44ce81ba,180107bd44ce81ba,From,Google My Business <googlemybusiness-noreply@g...
1,1801038fda1411c9,1801038fda1411c9,From,Best Buy 3-Day Sale <BestBuy@email.bestbuy.com>
2,180102f98edce8a3,180102f98edce8a3,From,Perraju Vegiraju via Lucidchart <sharing@luci...
3,1800fc7c726a611c,1800fc7c726a611c,From,Mindy Whitt via LinkedIn <messaging-digest-nor...
4,1800fc3241de9fc1,1800fc3241de9fc1,From,"""Audible.com"" <newsletters@audible.com>"
5,1800fac6b9e6213e,1800fac6b9e6213e,From,Udemy <no-reply@e.udemymail.com>
6,1800f82ef530ad18,1800f82ef530ad18,From,Udemy for Instructors <udemy@email.udemy.com>
7,1800ef465416ff55,1800ef465416ff55,From,Reddit <noreply@redditmail.com>
8,1800ed8d1a2c7aeb,1800ed8d1a2c7aeb,From,TechGig Latest News <technews@techgig.com>
9,1800ea9866509ef1,1800ea9866509ef1,From,TechGig <technews@techgig.com>


In [12]:
pd.read_sql('''
    WITH q AS (
        SELECT id, 
            threadid, 
            headers.name,
            headers.value,
            labelids
        FROM gmail_db.messages
        CROSS JOIN unnest(payload.headers) AS t(headers)
    ) SELECT * 
    FROM q
    WHERE name = 'From'
    LIMIT 10
''', conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,threadid,name,value,labelids
0,17ff16c72843659d,17ff16c72843659d,From,"""Priya Naruka (Partnerships, Turing)"" <priya@t...","[UNREAD, CATEGORY_UPDATES, INBOX]"
1,17ff0ccf34075b1e,17ff0ccf34075b1e,From,Mert Damlapinar via LinkedIn <newsletters-nore...,"[UNREAD, CATEGORY_UPDATES, INBOX]"
2,17ff0b06a91ba458,17ff0b06a91ba458,From,"""service@paypal.com"" <service@paypal.com>","[UNREAD, CATEGORY_UPDATES, INBOX]"
3,17ff09d8910bf133,17ff09d8910bf133,From,Snowflake Events <info@reply.snowflake.com>,"[CATEGORY_PROMOTIONS, UNREAD, INBOX]"
4,17ff083fa2df595e,17ff083fa2df595e,From,Kumaran Periaswamy via PayPal <service@paypal....,"[UNREAD, CATEGORY_UPDATES, INBOX]"
5,17ff0769b32cfdd3,17ff0769b32cfdd3,From,Henning Schwinum via LinkedIn <newsletters-nor...,"[UNREAD, CATEGORY_UPDATES, INBOX]"
6,17ff05552e91377d,17ff05552e91377d,From,Yaredi Eninga via LinkedIn <messaging-digest-n...,"[UNREAD, CATEGORY_UPDATES, INBOX]"
7,17ff05049c5f1ddb,17ff05049c5f1ddb,From,"""Amazon.com"" <store-news@amazon.com>","[CATEGORY_PROMOTIONS, UNREAD, INBOX]"
8,17ff04879e1d5bfe,17ff04879e1d5bfe,From,Udemy <no-reply@e.udemymail.com>,"[UNREAD, CATEGORY_UPDATES, INBOX]"
9,17ff040429cb0df7,17ff040429cb0df7,From,Scribd Editors <hello@hello.scribd.com>,"[CATEGORY_PROMOTIONS, UNREAD, INBOX]"


In [13]:
pd.read_sql('''
    WITH q AS (
        SELECT id, 
            threadid, 
            headers.name,
            headers.value,
            labelid
        FROM gmail_db.messages
        CROSS JOIN unnest(payload.headers) AS t(headers)
        CROSS JOIN unnest(labelids) AS t(labelid)
    ) SELECT * 
    FROM q
    WHERE name = 'From'
    LIMIT 10
''', conn)

/Users/itversity/Projects/Internal/bootcamp/gmail-puller/gp-venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,threadid,name,value,labelid
0,1803488b145538f8,1803488b145538f8,From,Uber Eats <noreply@uber.com>,CATEGORY_PROMOTIONS
1,1803488b145538f8,1803488b145538f8,From,Uber Eats <noreply@uber.com>,UNREAD
2,1803488b145538f8,1803488b145538f8,From,Uber Eats <noreply@uber.com>,INBOX
3,180347860980f2ed,180347860980f2ed,From,Santosh Kumar Thammineni via LinkedIn <messagi...,UNREAD
4,180347860980f2ed,180347860980f2ed,From,Santosh Kumar Thammineni via LinkedIn <messagi...,CATEGORY_UPDATES
5,180347860980f2ed,180347860980f2ed,From,Santosh Kumar Thammineni via LinkedIn <messagi...,INBOX
6,180342b144fe6b2e,180342b144fe6b2e,From,Best Buy Outlet <BestBuy@email.bestbuy.com>,CATEGORY_PROMOTIONS
7,180342b144fe6b2e,180342b144fe6b2e,From,Best Buy Outlet <BestBuy@email.bestbuy.com>,UNREAD
8,180342b144fe6b2e,180342b144fe6b2e,From,Best Buy Outlet <BestBuy@email.bestbuy.com>,INBOX
9,18033aba62e1e5ae,18033aba62e1e5ae,From,Medium <noreply@medium.com>,UNREAD
